In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'MSAC_1'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')



dataset = load_dataset('AbderrahmanSkiredj1/MSAC_darija_sentiment_analysis')
df = pd.DataFrame(dataset['train']) 

      
display(df.columns)
display(len(df))
display(df[:4])

df['label'] = df['label'].replace(',ne', 'neg')

classes = set(df['label'].values)
display(classes)

c = df['label'].value_counts()
display(c)

df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

df = df[['text', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


# ds = Dataset.from_pandas(df)
# ds = ds.train_test_split(test_size=0.2)

# display(ds)

max_sequence_length = 128


models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',
]





seeds = [0, 1, 42]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
    
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['text'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True) # , batched=True
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)  # , batched=True
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 25
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 20, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 20
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2024-09-17 04:54:18.916753: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 04:54:18.939674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 04:54:19.321752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Found cached dataset parquet (/home/ffq/.cache/huggingface/datasets/AbderrahmanSkiredj1___parquet/AbderrahmanSkiredj1--MSAC_darija_sentiment_analysis-78ec287cfc3da3ad/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)

  0%|          | 0/1 [00:00<?, ?it/s]

Index(['text', 'label'], dtype='object')

2000

,text,label
0,'طوال حياتي لم المس اي تغير حتى قدمت هذه الحكومة فل نقف بجانها بصوتنا',pos
1,'منتوج رائع وثمن مناسب ....جميل',pos
2,'كلنا ابن كيران لمتافق معايا يدير جيم',pos
3,'وفقك الله لولاية اخرى حقاش مكينش محسن منك',pos


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
20,0.434200,0.265469,0.912500,0.911477
40,0.208500,0.176375,0.942500,0.942341
60,0.125700,0.270741,0.930000,0.929984
80,0.110000,0.180492,0.945000,0.944767
100,0.071100,0.182479,0.957500,0.957383
120,0.034400,0.197984,0.952500,0.952314
140,0.014400,0.314507,0.945000,0.944729
160,0.008800,0.310951,0.935000,0.934959
180,0.017000,0.395095,0.932500,0.932037
200,0.000900,0.323503,0.950000,0.949939


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.9375,0.937359
3,UBC-NLP/MARBERT,0.9500,0.949955
6,UBC-NLP/MARBERTv2,0.9575,0.957383
9,aubmindlab/bert-base-arabertv02-twitter,0.9400,0.939878
15,qarib/bert-base-qarib,0.9525,0.952414
